In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Audio, display
from ipywidgets import FileUpload, Output, HBox, VBox, Layout
from pathlib import Path

In [2]:
sounds = {1: None, 2: None} 

def handle_upload(index, change, player):
    #lobal sounds
    name = change['new'][0]['name']
    content= change['new'][0]['content']
    file_path = f"samples/{name}"
    if not Path(file_path).is_file():
        with open(file_path, 'wb') as f:
            f.write(content)
    with player:
        player.clear_output()
        print(file_path)
        display(Audio(file_path))
        y, sr = librosa.load(file_path)
        sounds[index] = (y, sr)
        fig, axis = plt.subplots(4,1)
        fig.set_size_inches(10, 20)
        plot_sound(y, sr, axis[0])
        plot_spect(y, sr, fig, axis[1])    
        plot_autocorr_tempo(y, sr, fig, axis[2], axis[3])
        plt.show()
        

def plot_sound(y, sr, ax):   
    # Plot the array
    librosa.display.waveshow(y, sr=sr, ax=ax)
    
    # Set the title and labels of the plot
    ax.set_title("Line Plot of Array")
    #plt.xlabel("Time")
    #plt.ylabel("Value")
    
    # Show the plot
    #plt.show()


def plot_spect(y, sr, fig, ax):
    
    # Compute the spectrogram
    spectrogram = librosa.stft(y, hop_length=1024)

    spectrogram = np.abs(spectrogram)
    
    # Convert the spectrogram to dB scale
    spectrogram_db = librosa.amplitude_to_db(spectrogram, ref=np.max)
    
    # You can also visualize the spectrogram using librosa.display.specshow
    img = librosa.display.specshow(spectrogram_db, sr=sr, hop_length=1024, y_axis='log', x_axis='time', ax=ax)
    ax.set(title="Log-frequency Power Spectrogram")
    ax.label_outer()
    #plt.show()


def plot_autocorr_tempo(y, sr, fig, ax1, ax2):
    hl = 512
    onset = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hl)
    tempo = librosa.feature.tempogram(onset_envelope=onset, sr=sr, hop_length=hl)
    ax1.plot(librosa.times_like(onset), onset, label='Onset strength')
    ax1.legend(frameon=True)
    ax1.label_outer()
    librosa.display.specshow(tempo, sr=sr, hop_length=hl, x_axis='time', y_axis='tempo', cmap='magma', ax=ax2)
    ax2.set(title='Autocorrelation tempogram')

    

In [3]:
upload1, upload2 = FileUpload(multiple=False), FileUpload(multiple=False)
player1, player2 = Output(), Output()
#summary = Output()

upload1.observe(lambda ch: handle_upload(1, ch, player1), names='value')
upload2.observe(lambda ch: handle_upload(2, ch, player2), names='value')

boxes = HBox([
    VBox([upload1, player1], layout= Layout(width='50%')),
    VBox([upload2, player2], layout= Layout(width='50%'))
])

display(boxes)

In [8]:
def calc_tempo(y, sr):
    return round(librosa.feature.tempo(y=y, sr=sr)[0], 1)


def calc_spec_center(y, sr):
    scs = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    return round(np.mean(abs(scs)))


def calc_spec_band(y, sr):
    scs = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    return round(np.mean(abs(scs)))


def compare_sounds(y1, sr1, y2, sr2):
    tempo_diff = abs(round(calc_tempo(y1, sr1) - calc_tempo(y2, sr2)))
    spec_center_diff = abs(calc_spec_center(y1, sr1) - calc_spec_center(y2, sr2))
    spec_band_diff = abs(calc_spec_band(y1, sr1) - calc_spec_band(y2, sr2))
    print(f"Tempo difference: {tempo_diff}")
    print(f"Spectral centroid difference: {spec_center_diff}")
    print(f"Spectral bandwidth difference: {spec_band_diff}")
    
if sounds[1] and sounds[2]:        
    print("Summary:")
    y1, sr1 = sounds[1]
    y2, sr2 = sounds[2]
    compare_sounds(y1, sr1, y2, sr2)

Summary:
Tempo difference: 34
Spectral centroid difference: 370
Spectral bandwidth difference: 426


In [ ]:
# y, sr = librosa.load(file_path)
# fig, axis = plt.subplots(2,1)
# fig.set_size_inches(10, 15)
# plot_sound(y, sr, axis[0])
# plot_spect(y, sr, fig, axis[1])        
# plt.show()

In [ ]:
cc

In [ ]:
#filename = librosa.example('nutcracker')
#plot_spect(filename)